In [21]:
import os
from IPython.display import display
import ipywidgets as widgets
from PIL import Image
from io import BytesIO

def list_png_files(base_path, subdirectory="", png_name=""):
    """List PNG files in the directory and subdirectories, optionally filtered by subdirectory and PNG name."""
    png_files = []
    target_path = os.path.join(base_path, subdirectory) if subdirectory else base_path
    for root, dirs, files in os.walk(target_path):
        for file in files:
            if file.endswith(".png") and (png_name.lower() in file.lower() or not png_name):
                png_files.append(os.path.join(root, file))
    return png_files

def resize_image(image_path, max_width=800, max_height=600):
    """Resize the image to fit within specified dimensions, maintaining aspect ratio."""
    with Image.open(image_path) as img:
        img.thumbnail((max_width, max_height), Image.Resampling.LANCZOS)
        with BytesIO() as buf:
            img.save(buf, format='PNG')
            image_widget = widgets.Image(value=buf.getvalue(), format='png', width=img.width, height=img.height)
    return image_widget, image_path

def browse_png_files(folder_path):
    """Create widgets to browse through PNG files in the given folder, with subdirectory and file name filtering."""
    base_path = folder_path
    output = widgets.Output()
    subdirectory_input = widgets.Text(value='', placeholder='Type subdirectory name', description='Subdirectory:', disabled=False)
    png_name_input = widgets.Text(value='', placeholder='Type PNG name', description='PNG Name:', disabled=False)
    
    def update_image_list(subdirectory="", png_name=""):
        """Update the list of PNG files based on the specified subdirectory and PNG name."""
        return list_png_files(base_path, subdirectory, png_name)
    
    def view_image(change):
        """Display the selected image from the updated list."""
        with output:
            output.clear_output(wait=True)
            png_files = update_image_list(subdirectory_input.value, png_name_input.value)
            if not png_files:
                display(widgets.HTML(value="<b>No PNG files found matching the criteria.</b>"))
                return
            if slider.value >= len(png_files):
                slider.value = 0  # Reset slider if out of range
            image_widget, file_path = resize_image(png_files[slider.value])
            display(widgets.HTML(value=f"<b>File Path:</b> {file_path}"))
            display(image_widget)
    
    slider = widgets.IntSlider(value=0, min=0, max=0, step=1, description='Image Index:')
    slider.observe(view_image, names='value')
    
    def on_text_change(change):
        """Handle updates to the text inputs for subdirectory or PNG name filtering."""
        png_files = update_image_list(subdirectory_input.value, png_name_input.value)
        slider.max = len(png_files) - 1  # Update slider range
        view_image(None)
    
    subdirectory_input.observe(on_text_change, names='value')
    png_name_input.observe(on_text_change, names='value')
    
    # Display the widgets
    display(subdirectory_input, png_name_input, slider, output)
    view_image(None)  # Initial call to display the first image without filters

# Example usage
entropy = 2
output = 2
attack = 6
browse_png_files(f"./results/stationary_distribution/robustness_analysis/entropy_{entropy}/output_{output}/diversity/attack_{attack}")


Text(value='', description='Subdirectory:', placeholder='Type subdirectory name')

Text(value='', description='PNG Name:', placeholder='Type PNG name')

IntSlider(value=0, description='Image Index:', max=0)

Output()